#Modelo de Machine Learning de Ponta a Ponta

Depois que apresentamos as soluções para melhorar o desempenho da empresa e recuperar uma parte do faturamento, estamos prontos para criar um Modelo de Machine Learning capaz de prever a demanda para os próximos dias, gerando valor para os parceiros e levando informações preciosas para que eles possam se preparar para o dia de trabalho.

In [ ]:
#import das bibliotecas
import pandas as pd
import numpy as np

## Análise de Estrutura

## Remoção de Outliers

In [ ]:
#Leitura do csv
df_orders = pd.read_csv("orders.csv")

In [ ]:
#Corte de outliers
df_orders = df_orders[(df_orders['order_amount'] >= 15) &
          (df_orders['order_amount'] <= 200)]

In [ ]:
#info
df_orders.info()

## Novas Features

In [ ]:
#converter order_moment_created para data
df_orders['order_moment_created'] = pd.to_datetime(df_orders['order_moment_created'])

In [ ]:
#nova coluna com o dia da semana
df_orders['day_of_week'] = df_orders['order_moment_created'].dt.day_of_week

In [ ]:
#Análise da quantidade de horas que temos pedidos por dia
df_orders['order_created_hour'].value_counts() \
  .reset_index() \
  .sort_values('order_created_hour')

In [ ]:
#criando função por faixa de horário
def faixa_horario(hora):
  if hora >= 0 and hora <= 5:
    return 'madrugada'
  elif hora >= 6 and hora <= 10:
    return 'manha'
  elif hora >= 11 and hora <= 14:
    return 'almoco'
  elif hora >= 15 and hora <= 18:
    return 'tarde'
  else:
    return 'noite'

faixa_horario(19)

In [ ]:
#aplicando função e criando uma nova coluna com o nome de faixa de horário
df_orders['faixa_horario'] = df_orders['order_created_hour'].apply(faixa_horario)
df_orders['faixa_horario'].value_counts()

## DataPrep

In [ ]:
#Criando um novo dataset de orders_treatment
df_orders_tratamento = df_orders.copy()

In [ ]:
#info
df_orders_tratamento.info()

### Data Cleaning

#### Exclusão de colunas

In [ ]:
#após realizar análise das variáveis excluir variável order_moment_delivered
df_orders_tratamento.drop('order_moment_delivered', axis=1, inplace=True)

In [ ]:
#definição das colunas que serão excluídas da nossa base
columns_delete = ['payment_order_id',
'delivery_order_id',
'order_created_minute',
'order_created_month',
'order_created_year',
'order_moment_created',
'order_moment_accepted',
'order_moment_ready',
'order_moment_collected',
'order_moment_in_expedition',
'order_moment_delivering',
'order_moment_finished',
'order_delivery_fee',
'order_delivery_cost',
'order_created_hour']

In [ ]:
#excluir colunas
df_orders_tratamento.drop(columns_delete, axis=1, inplace=True)

#### Prenchendo valores nulos

In [ ]:
#info
df_orders_tratamento.info()

In [ ]:
#utilização do fillna com a mediana para preencher os valores do order_metric_collected_time
df_orders_tratamento['order_metric_collected_time'] \
  .fillna(df_orders_tratamento['order_metric_collected_time'].median(), inplace=True)

In [ ]:
df_orders_tratamento.info()

### Criação do dataset final

In [ ]:
#Criação do dataset final de grupos
"""df_orders_treatment_group = df_orders_tratamento.groupby(['store_id', 'channel_id', 'order_created_day', 'day_of_week', 'faixa_horario']) \
  .agg({'order_metric_collected_time':'median',
        'order_metric_paused_time':'median',
        'order_metric_production_time':'median',
        'order_metric_walking_time':'median',
        'order_metric_expediton_speed_time':'median',
        'order_metric_transit_time':'median',
        'order_metric_cycle_time':'median',
        'order_id':'count'}) \
  .reset_index() \
  .sort_values('order_id', ascending=False)

df_orders_treatment_group=df_orders_treatment_group.rename(columns = {'order_id':'demanda'})

df_orders_treatment_group.head()"""

In [ ]:
#info
df_orders_tratamento.info()

### Criando um conjunto de testes

Antes de seguir adiante, vamos precisar criar um conjunto de teste, colocá-lo de lado e nunca checá-lo. <br/>

Quando estimamos o erro de generalização utilizando o conjunto de teste, sua estimativa será muito otimista e será lançado um sistema que não funcionará tão bem quanto o esperado.  <br/>

Isso é chamado de **data snooping bias.** <br/>

O Scikit-Learn fornece algumas funções para dividir conjuntos de dados em vários subconjuntos de diversas maneiras. A função mais simples é train_test_split.

In [ ]:
#import do train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
#Separa a base de treino e teste
train_set, test_set = train_test_split(df_orders_tratamento,
                                      test_size=0.2,
                                      random_state=42)

df_orders_train = train_set.copy()

In [ ]:
#Separa a Label principal
df_orders_treatment_label = df_orders_train[['order_status']].copy()

In [ ]:
#Remove a Label da base
df_orders_treatment = df_orders_train.drop('order_status', axis=1)

In [ ]:
#Seleciona as variáveis float64
df_float_64 = df_orders_treatment.select_dtypes(np.float64).copy()

In [ ]:
#Import do SimpleImputer
from sklearn.impute import SimpleImputer

In [ ]:
#Adicionando estratégia da mediana no imputer
imputer = SimpleImputer(strategy='median')

In [ ]:
#Treina o imputer
imputer.fit(df_float_64)

In [ ]:
#Cria nova matriz preenchendo os valores nulos com o imputer
float_vars = imputer.transform(df_float_64)
#float_vars

In [ ]:
#Cria novo dataset para apresentar os valores
df_64 = pd.DataFrame(float_vars, columns=df_float_64.columns,
                     index=df_float_64.index)
df_64.info()

## Seleção Final

## Variáveis Categóricas

In [ ]:
#info
df_orders_treatment.info()

In [ ]:
#seleção de day_of_week e faixa_horario
df_orders_treatment[['day_of_week', 'faixa_horario']]

In [ ]:
#value_counts faixa de horario
df_orders_treatment['faixa_horario'].value_counts()

In [ ]:
#novo dataset com a seleção das variaveis numéricas categóricas
df_orders_treatment_cat_num = df_orders_treatment[['store_id',
                                                   'channel_id',
                                                   'order_created_day',
                                                   'day_of_week']].copy()

In [ ]:
#novo dataset com a seleção das variáveis categóricas
df_orders_treatment_cat = df_orders_treatment[['faixa_horario']].copy()

In [ ]:
#import do OrdinalEncoder e do OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

### Ordinal Encoder
A maioria dos algoritmos de Aprendizado de Máquina prefere trabalhar com números, então vamos converter as categorias de texto para números. Para tanto, podemos utilizar o método OrdinalEncoder(), que mapeia cada categoria para um número inteiro diferente.

In [ ]:
#tratamento com o OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_orders_treatment_cat_encoded = ordinal_encoder.fit_transform(df_orders_treatment_cat)
#df_orders_treatment_cat_encoded[:10]
ordinal_encoder.categories_

### OneHotEncoder
O OrdinalEncoder pega a quantidade de atributos e converte em números, porém transformando puramente em números ele cria uma diferença de valores entre os números. As categorias 0 e 1 transformadas tem uma distância semelhante, não podemos dizer o mesmo para as categorias 0 e 2, os algoritmos de ML enxergarão essa escala como uma diferença significa entre os dados. <br/>
A utilização do OneHotEncoder é melhor aproveitada para esses casos. Ela cria novos atributos de acordo com a quantidade de atributos com 0 e 1.


In [ ]:
#tratamento da categoria com o OneHotEncoder
cat_encoder = OneHotEncoder()
df_orders_treatment_cat_1hot = cat_encoder.fit_transform(df_orders_treatment_cat)
cat_encoder.categories_

In [ ]:
#Visuaulização das categorias em array
df_orders_treatment_cat_1hot.toarray()

In [ ]:
#Visualização das categorias
cat_encoder.categories_

## Escalonando nossos dados

In [ ]:
#visualização do df_float64
df_64.head()

In [ ]:
#import do MinMaxScaler e do StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### MinMaxScaler
O escalonamento min-max (muitas pessoas chamam de normalização) é bastante simples: os valores são deslocados e redimensionados para que acabem variando de 0 a 1. Ele subtrai o valor mínimo e divide pelo máximo menos o mínimo. O Scikit-Learn fornece um transformador chamado MinMaxScaler para isso. Ele possui um hiper parâmetro feature_range que permite alterar o intervalo se não quiser 0-1 por algum motivo.

In [ ]:
#Tratamento com MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_64)
df_float64_transform = scaler.transform(df_64)

In [ ]:
#Visualiza transformação
df_float64_transform

### StandardScaler
A padronização é bem diferente: em primeiro lugar ela subtrai o valor médio (assim os valores padronizados sempre têm média zero) e, em seguida, divide pela variância, de modo que a distribuição resultante tenha variância unitária.
Ao contrário do escalonamento min-max, a padronização não vincula valores a um intervalo específico, o que pode ser um problema para alguns algoritmos.
No entanto, a padronização é muito menos afetada por outliers.
O Scikit-Learn fornece um transformador para padronização chamado StandardScaler.

In [ ]:
#Tratamento com o SatandardScaler
standard = StandardScaler()
standard.fit(df_64)
df_float64_transform_standard = standard.transform(df_64)

In [ ]:
#Visualiza transformação
df_float64_transform_standard

**Quando usar cada um?**<br/>
- MinMax Scaling é indicado quando a distribuição dos dados não segue uma gaussiana ou quando você sabe que as variáveis devem estar em um intervalo específico (ex.: redes neurais com funções de ativação como sigmoid).
- Standard Scaling é mais indicado quando os dados têm uma distribuição aproximadamente normal e é necessário manter essa forma de distribuição.

## Pipeline

In [ ]:
#import do Pipeline
from sklearn.pipeline import Pipeline

Existem muitas etapas de transformação de dados que precisam ser executadas na ordem correta. Felizmente, o Scikit-Learn fornece a classe Pipeline para ajudar tais sequências de transformações.

O construtor Pipeline se vale de uma lista de pares de nome/estimador que definem uma sequência de etapas. Todos, exceto o último estimador, devem ser transformadores (ou seja, eles devem ter um método fit_transform()).


### Pipeline Numérico

In [ ]:
#Criação do novo pipeline com o Imputer e StandardScaler
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

In [ ]:
#Criação de um novo dataset com o pipline numérico
orders_num_tr = num_pipeline.fit_transform(df_float_64)

In [ ]:
#Apresentação do Shape da transformação após o pipeline
orders_num_tr.shape

### Pipeline Categório + Full Pipeline

In [ ]:
#Import do column transform
from sklearn.compose import ColumnTransformer

In [ ]:
#Seleção das variáveis numéricas, categóricas numéricas e categóricas.
num_attr = list(df_float_64.columns)
cat_num_attr = list(df_orders_treatment_cat_num.columns)
cat_attr = list(df_orders_treatment_cat.columns)

In [ ]:
#Criação do Full Pipeline
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attr),
    ('cat_num', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_num_attr),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_attr)
])

In [ ]:
#Transformação dos dados através do full pipeline
df_orders_prepared = full_pipeline.fit_transform(df_orders_treatment)
df_orders_prepared

In [ ]:
df_orders_treatment_label

#Classificação

##SGDClassifier

O SGDClassifier (Stochastic Gradient Descent Classifier) é um algoritmo de aprendizado supervisionado, principalmente usado para tarefas de classificação, mas também pode ser aplicado para regressão. Ele é parte da biblioteca scikit-learn e é especialmente útil em problemas de aprendizado com grandes volumes de dados, pois realiza atualizações de parâmetros gradualmente em vez de carregar todo o conjunto de dados na memória de uma só vez.

### **SGDClassifier: Fórmulas e Explicação**

O `SGDClassifier` aplica o Gradiente Estocástico para minimizar uma função de custo associada ao modelo escolhido, como a **SVM Linear** ou a **Regressão Logística**.

### 1. SVM Linear (Função de Custo Hinge)

Para uma SVM Linear, o `SGDClassifier` minimiza a função de custo **hinge**, que maximiza a margem entre as classes:

$$
L(w, b) = \frac{1}{n} \sum_{i=1}^n \max(0, 1 - y_i (w \cdot x_i + b)) + \frac{\alpha}{2} \|w\|^2
$$

Onde:
- $( w $): vetor de pesos (ou coeficientes) do modelo.
- $( b $): bias (intercepto).
- $( x_i $): vetor de características da amostra $( i $).
- $( y_i $): rótulo verdadeiro da amostra $( i $) (normalmente +1 ou -1 para SVM).
- $( n $): número de amostras no lote.
- $( \alpha $): parâmetro de regularização, que controla o impacto da penalidade $( \|w\|^2 $) para evitar overfitting.

A função hinge aplica uma penalidade apenas para amostras que estão incorretamente classificadas ou muito próximas da margem de decisão.


In [ ]:
#import do SGDCLassifier
from sklearn.linear_model import SGDClassifier

#Treinamento do Modelo
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(df_orders_prepared, df_orders_treatment_label)

In [ ]:
#Seleciona os 10 primeiros registros de dados e labels
dados = df_orders_treatment.iloc[:10]
labels = df_orders_treatment_label.iloc[:10]

#Passa pelo full pipeline
dados_preparados = full_pipeline.transform(dados)

In [ ]:
#Realiza a predição dos valores
sgd_clf.predict(dados_preparados)

In [ ]:
#Verifica as Labels
labels.values

### Validação Cruzada (cross_val_score)

Utilizamos o cross_val_score em problemas de classificação para obter uma avaliação mais robusta e confiável do desempenho do modelo. Ele realiza a validação cruzada, que divide o conjunto de dados em múltiplos subconjuntos (ou folds) e testa o modelo em diferentes partições dos dados, ajudando a verificar a capacidade de generalização do modelo para dados novos.

**A Validação Cruzada Reduz o Overfitting**: Em vez de treinar e testar o modelo em uma única divisão dos dados, a validação cruzada usa várias divisões, minimizando o risco de overfitting. Com isso, evitamos ajustar o modelo apenas a uma fração específica dos dados.



In [ ]:
#import do cross_val_score
from sklearn.model_selection import cross_val_score
#Realiza o cross validation com 3 folds e orientado pela "accuracy"
cross_val_score(sgd_clf, df_orders_prepared, df_orders_treatment_label, cv=3, scoring="accuracy")

### Validação Cruzada de Predição

O cross_val_predict também utiliza a validação cruzada, mas, em vez de retornar uma pontuação média como o cross_val_score, ele gera diretamente as predições do modelo para cada amostra do conjunto de dados.

Em resumo, o cross_val_predict é útil quando queremos:

**Obter Predições de Validação Cruzada:** Ele retorna uma matriz com as predições do modelo para cada amostra no conjunto de dados, permitindo que analisemos o desempenho do modelo em nível de amostra.

**Análise Detalhada de Métricas:** Como temos as predições para cada amostra, podemos calcular métricas como precisão, recall, F1-score, curva ROC e AUC, utilizando todas as amostras do conjunto de dados como se fossem dados de teste, sem afetar o processo de treino.

**Redução de Overfitting em Métricas de Avaliação:** Calculando as métricas a partir das predições feitas por cross_val_predict, conseguimos uma avaliação mais próxima de um cenário real, onde cada predição foi feita com dados de treino diferentes dos dados de teste, evitando o uso de um conjunto fixo de treino e teste.

In [ ]:
#import do cross_val_predict
from sklearn.model_selection import cross_val_predict
#realiza a predição através do cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, df_orders_prepared, df_orders_treatment_label, cv=3)

### Matriz de Confusão

A matriz de confusão é uma ferramenta muito útil para avaliar o desempenho de um modelo de classificação. Ela mostra, em uma tabela, as predições feitas pelo modelo versus os valores reais, ajudando a identificar erros específicos em cada classe e a entender onde o modelo está acertando ou errando.

**Estrutura da Matriz de Confusão**
Para um problema de classificação binária, a matriz de confusão é normalmente organizada da seguinte forma:


\begin{array}{|c|c|c|}
\hline
 & \text{Previsão Positiva} & \text{Previsão Negativa} \\
\hline
\text{Classe Positiva} & \text{True Positive (TP)} & \text{False Negative (FN)} \\
\hline
\text{Classe Negativa} & \text{False Positive (FP)} & \text{True Negative (TN)} \\
\hline
\end{array}

### Componentes da Matriz de Confusão

1. **True Positive (TP)**: Amostras corretamente classificadas como positivas.
2. **False Positive (FP)**: Amostras negativas incorretamente classificadas como positivas (**falso alarme**).
3. **True Negative (TN)**: Amostras corretamente classificadas como negativas.
4. **False Negative (FN)**: Amostras positivas incorretamente classificadas como negativas (**falta de detecção**).

#### Métricas Derivadas da Matriz de Confusão

A partir dos valores de TP, FP, TN e FN, podemos calcular várias métricas para avaliar o modelo:

1. **Acurácia**: Percentual de predições corretas em relação ao total de amostras.
   $
   \text{Acurácia} = \frac{TP + TN}{TP + TN + FP + FN}
   $

2. **Precisão**: Percentual de predições positivas que são realmente positivas.
   $
   \text{Precisão} = \frac{TP}{TP + FP}
   $

3. **Recall (ou Sensibilidade)**: Percentual de amostras positivas corretamente identificadas.
   $
   \text{Recall} = \frac{TP}{TP + FN}
   $

4. **F1-Score**: Média harmônica entre precisão e recall, útil quando as classes são desbalanceadas.
   $
   F1 = 2 \times \frac{\text{Precisão} \times \text{Recall}}{\text{Precisão} + \text{Recall}}
   $


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df_orders_treatment_label, y_train_pred)

### Classification Report

O `classification_report` do `sklearn` gera um relatório com métricas de desempenho de um modelo de classificação para cada classe, facilitando a análise e comparação do desempenho entre classes.

## Principais Métricas do `classification_report`

1. **Acurácia**: Percentual de predições corretas em relação ao total de amostras.
   $
   \text{Acurácia} = \frac{TP + TN}{TP + TN + FP + FN}
   $


2. **Precisão (Precision)**:
   - A precisão indica a porcentagem de predições positivas que são realmente positivas.
   - Fórmula:
     $
      \text{Precisão} = \frac{TP}{TP + FP}
      $

3. **Recall (Sensibilidade)**:
   - O recall mede a capacidade do modelo de identificar corretamente todas as amostras da classe positiva.
   - Fórmula:
     $
   \text{Recall} = \frac{TP}{TP + FN}
   $

4. **F1-Score**:
   - O F1-Score é a média harmônica entre precisão e recall, útil quando temos classes desbalanceadas.
   - Fórmula:
     $
   F1 = 2 \times \frac{\text{Precisão} \times \text{Recall}}{\text{Precisão} + \text{Recall}}
   $

5. **Suporte (Support)**:
   - O suporte é o número de ocorrências reais de cada classe no conjunto de dados, ajudando a avaliar o impacto de cada classe no cálculo das métricas.


In [ ]:
#import do classification_report
from sklearn.metrics import classification_report
#realizada o classification report
report = classification_report(df_orders_treatment_label, y_train_pred)
print(report)

In [ ]:
#Analisa a porcentagem entre status
df_orders_treatment_label['order_status'].value_counts(normalize=True)

### Desbalanceamento entre Classes

O desbalanceamento de classes pode afetar negativamente o desempenho de modelos de machine learning, pois o modelo pode aprender a favorecer a classe majoritária, ignorando as amostras da classe minoritária. Essas são umas das principais estratégias para lidar com classes desbalanceadas:

####Oversampling
O oversampling consiste em aumentar a quantidade de instâncias da classe minoritária, de forma que ela tenha uma representação mais equilibrada em relação à classe majoritária. O método mais simples de oversampling é a duplicação de dados da classe minoritária, mas isso pode levar a overfitting.

####Undersampling
O undersampling é o processo de reduzir a quantidade de instâncias da classe majoritária, diminuindo o número de amostras para equilibrá-lo com a classe minoritária. A técnica mais simples de undersampling é a remoção aleatória de instâncias da classe majoritária. No entanto, isso pode resultar na perda de informações importantes, principalmente se o conjunto de dados for pequeno.

## SMOTE (Synthetic Minority Over-sampling Technique)
O SMOTE é uma técnica de oversampling que cria novas instâncias sintéticas da classe minoritária em vez de duplicar instâncias. Ele funciona da seguinte maneira:

Para cada instância da classe minoritária, o SMOTE seleciona uma ou mais instâncias vizinhas próximas.
Novas instâncias são geradas interpolando os atributos entre a instância original e a vizinha selecionada.
Esse processo ajuda a evitar o overfitting, pois as novas amostras são únicas e não cópias diretas das amostras existentes.




In [ ]:
#import do SMOTE
from imblearn.over_sampling import SMOTE

In [ ]:
# criando uma instância do SMOTE
smote = SMOTE()

# balanceando os dados
X_resampled, y_resampled = smote.fit_resample(df_orders_prepared, df_orders_treatment_label)

In [ ]:
#Análise entre status
y_resampled['order_status'].value_counts()

In [ ]:
#Realizando o treino novamente com o SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_resampled, y_resampled)

In [ ]:
#Realizando a validação cruzada orientada a previsão pelo resampling do SMOTE
y_train_pred = cross_val_predict(sgd_clf, X_resampled, y_resampled, cv=3)

In [ ]:
#Análise do classification_report
from sklearn.metrics import classification_report
report = classification_report(y_resampled, y_train_pred)
print(report)

## RandomForestClassifier

In [ ]:
#Import do RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Treina o modelo no RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
forest_clf.fit(X_resampled, y_resampled)

In [ ]:
#Realiza a Validação Cruzada
y_train_pred = cross_val_predict(forest_clf, X_resampled, y_resampled, cv=3)

In [ ]:
#Print do Classification Report
from sklearn.metrics import classification_report
report = classification_report(y_resampled, y_train_pred)
print(report)

### Modelo de Classificação Final

In [ ]:
#Escolhe o modelo
final_model = forest_clf

In [ ]:
#Seleciona o dataset de teste
df_orders_test = test_set.copy()
x_test = df_orders_test.drop('order_status', axis=1)
y_test = df_orders_test['order_status'].copy()

#Faz o tratamento dos dados
x_test_prepared = full_pipeline.transform(x_test)
final_predictions = final_model.predict(x_test_prepared)

In [ ]:
#Analise do classification_report
from sklearn.metrics import classification_report
report = classification_report(y_test, final_predictions)
print(report)